In [28]:
from sqlalchemy import create_engine
import os
import numpy as np
import pandas as pd
from dotenv import load_dotenv

def create_connection():

    load_dotenv()
    host = os.environ.get('DB_DESTINATION_HOST')
    port = os.environ.get('DB_DESTINATION_PORT')
    db = os.environ.get('DB_DESTINATION_NAME')
    username = os.environ.get('DB_DESTINATION_USER')
    password = os.environ.get('DB_DESTINATION_PASSWORD')
    
    # print(f'postgresql://{username}:{password}@{host}:{port}/{db}')
    conn = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{db}')
    return conn

# устанавливаем соединение с базой
conn = create_connection()
orig_data = pd.DataFrame(pd.read_sql('select * from clean_users_churn', conn))
print(orig_data.head())
orig_data.dtypes

      id customer_id begin_date   end_date            type paperless_billing  \
0  28117  6993-OHLXR 2019-11-01 2020-01-01  Month-to-month                No   
1  28118  4250-FDVOU 2019-04-01 2019-10-01  Month-to-month                No   
2  28119  7284-ZZLOH 2016-05-01        NaT        Two year                No   
3  28120  5277-ZLOOR 2019-08-01 2019-10-01  Month-to-month               Yes   
4  28121  2141-RRYGO 2019-06-01 2019-10-01  Month-to-month               Yes   

              payment_method  monthly_charges  total_charges internet_service  \
0           Electronic check            68.95         119.75      Fiber optic   
1           Electronic check            56.25         389.10              DSL   
2    Credit card (automatic)            53.30        2296.25              DSL   
3           Electronic check            85.55         187.45      Fiber optic   
4  Bank transfer (automatic)            68.65         261.25              DSL   

   ... device_protection tech_su

id                            int64
customer_id                  object
begin_date           datetime64[ns]
end_date             datetime64[ns]
type                         object
paperless_billing            object
payment_method               object
monthly_charges             float64
total_charges               float64
internet_service             object
online_security              object
online_backup                object
device_protection            object
tech_support                 object
streaming_tv                 object
streaming_movies             object
gender                       object
senior_citizen                int64
partner                      object
dependents                   object
multiple_lines               object
target                        int64
dtype: object

In [19]:
data=orig_data.copy()

In [20]:
data.drop(columns=['id', 'customer_id', 'begin_date', 'end_date'], inplace=True)
data

,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,Month-to-month,No,Electronic check,68.95,119.75,Fiber optic,No,No,No,No,No,No,Female,0,No,No,No,1
1,Month-to-month,No,Electronic check,56.25,389.10,DSL,Yes,Yes,No,No,No,No,Female,0,No,No,No,1
2,Two year,No,Credit card (automatic),53.30,2296.25,DSL,Yes,No,Yes,No,No,No,Male,0,Yes,No,No,0
3,Month-to-month,Yes,Electronic check,85.55,187.45,Fiber optic,No,No,No,No,Yes,No,Female,1,No,No,Yes,1
4,Month-to-month,Yes,Bank transfer (automatic),68.65,261.25,DSL,No,Yes,No,No,Yes,Yes,Female,0,No,No,No,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7003,Month-to-month,Yes,Mailed check,20.05,39.25,Fiber optic,No,No,No,No,No,No,Female,0,No,No,No,0
7004,One year,No,Credit card (automatic),60.00,3316.10,DSL,Yes,Yes,No,No,No,No,Male,1,Yes,No,Yes,0
7005,Month-to-month,Yes,Electronic check,75.75,75.75,Fiber optic,No,No,No,No,No,No,Male,1,No,No,Yes,1
7006,Month-to-month,Yes,Credit card (automatic),69.50,2625.25,Fiber optic,No,No,No,No,No,No,Male,0,No,No,No,0


In [13]:
cat_features = data.select_dtypes(include='object')
potential_binary_features = cat_features.nunique() == 2

binary_cat_features = cat_features[potential_binary_features[potential_binary_features].index]
other_cat_features = cat_features[potential_binary_features[~potential_binary_features].index]
num_features = data.select_dtypes(['float'])

In [14]:
from sklearn.preprocessing import OneHotEncoder

one_hot_drop = OneHotEncoder(drop='if_binary', sparse_output=False) 

In [15]:
drop_res = one_hot_drop.fit_transform(binary_cat_features)
drop_res = pd.DataFrame(drop_res, columns=one_hot_drop.get_feature_names_out())
print(drop_res.head())
print('shape: ', drop_res.shape) 

   paperless_billing_Yes  internet_service_Fiber optic  online_security_Yes  \
0                    0.0                           1.0                  0.0   
1                    0.0                           0.0                  1.0   
2                    0.0                           0.0                  1.0   
3                    1.0                           1.0                  0.0   
4                    1.0                           0.0                  0.0   

   online_backup_Yes  device_protection_Yes  tech_support_Yes  \
0                0.0                    0.0               0.0   
1                1.0                    0.0               0.0   
2                0.0                    1.0               0.0   
3                0.0                    0.0               0.0   
4                1.0                    0.0               0.0   

   streaming_tv_Yes  streaming_movies_Yes  gender_Male  partner_Yes  \
0               0.0                   0.0          0.0         

In [16]:
from category_encoders import CatBoostEncoder

catboost_enc = CatBoostEncoder()

catboost_enc.fit_transform(other_cat_features, data['target']).head() 

,type,payment_method
0,0.265839,0.265839
1,0.632920,0.632920
2,0.265839,0.265839
3,0.755280,0.755280
4,0.816460,0.265839


In [17]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler_res = scaler.fit_transform(num_features)
print(scaler_res)
print(pd.DataFrame(scaler_res, columns=scaler.get_feature_names_out())) 

[[ 0.13404913 -0.95792733]
 [-0.28872075 -0.83909486]
 [-0.3869232   0.00230606]
 ...
 [ 0.3604141  -0.97733935]
 [ 0.15235806  0.14745505]
 [ 1.26587399  2.02733304]]
      monthly_charges  total_charges
0            0.134049      -0.957927
1           -0.288721      -0.839095
2           -0.386923       0.002306
3            0.686646      -0.928059
4            0.124062      -0.895500
...               ...            ...
7003        -1.493781      -0.993443
7004        -0.163887       0.452246
7005         0.360414      -0.977339
7006         0.152358       0.147455
7007         1.265874       2.027333

[7008 rows x 2 columns]


In [21]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from category_encoders import CatBoostEncoder

binary_cols = binary_cat_features.columns.tolist()
non_binary_cat_cols = other_cat_features.columns.tolist()
num_cols = num_features.columns.tolist()

# определите список трансформаций в рамках ColumnTransformer
preprocessor = ColumnTransformer(
		[
            ('binary', OneHotEncoder(drop='if_binary'), binary_cols),
            ('other_cat', CatBoostEncoder(), non_binary_cat_cols),
            ('scaler', StandardScaler(), num_cols),
        ],
    verbose_feature_names_out=False
)

# трансформируйте исходные данные data с помощью созданного preprocessor
data_transformed = preprocessor.fit_transform(data, data['target'])
print(data_transformed)
print(pd.DataFrame(data_transformed, columns=preprocessor.get_feature_names_out()))

[[ 0.          1.          0.         ...  0.26583904  0.13404913
  -0.95792733]
 [ 0.          0.          1.         ...  0.63291952 -0.28872075
  -0.83909486]
 [ 0.          0.          1.         ...  0.26583904 -0.3869232
   0.00230606]
 ...
 [ 1.          1.          0.         ...  0.4516162   0.3604141
  -0.97733935]
 [ 1.          1.          0.         ...  0.15214858  0.15235806
   0.14745505]
 [ 1.          1.          1.         ...  0.15204855  1.26587399
   2.02733304]]
      paperless_billing_Yes  internet_service_Fiber optic  \
0                       0.0                           1.0   
1                       0.0                           0.0   
2                       0.0                           0.0   
3                       1.0                           1.0   
4                       1.0                           0.0   
...                     ...                           ...   
7003                    1.0                           1.0   
7004                  

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from category_encoders import CatBoostEncoder

X_tr, X_val, y_tr, y_val = train_test_split(data, data['target'], stratify=data['target'])
data.head()
preprocessor = ColumnTransformer(
    [
    ('binary', OneHotEncoder(drop='if_binary'), binary_cols),
    ('cat', CatBoostEncoder(), non_binary_cat_cols),
    ('num', StandardScaler(), num_cols)
    ],
    remainder='drop',
    verbose_feature_names_out=False
)
model = CatBoostClassifier(auto_class_weights='Balanced')

# создайте пайплайн
pipeline = Pipeline(
	# ваш код здесь #
    [
        ('transformer', preprocessor),
        ('model', model)
    ]
)

# обучите пайплайн
# ваш код здесь #
pipeline.fit(X_tr, y_tr)

# получите предсказания для тестовой выборки
# ваш код здесь #
y_pred=pipeline.predict(X_val)

print('f1:', f1_score(y_val, y_pred))
print('roc_auc:', roc_auc_score(y_val, y_pred))

Learning rate set to 0.020924
0:	learn: 0.6840914	total: 54.1ms	remaining: 54s
1:	learn: 0.6758409	total: 57.9ms	remaining: 28.9s
2:	learn: 0.6673365	total: 64.7ms	remaining: 21.5s
3:	learn: 0.6588751	total: 71.3ms	remaining: 17.8s
4:	learn: 0.6519727	total: 79.5ms	remaining: 15.8s
5:	learn: 0.6452648	total: 86.4ms	remaining: 14.3s
6:	learn: 0.6389021	total: 94.1ms	remaining: 13.3s
7:	learn: 0.6333099	total: 97.4ms	remaining: 12.1s
8:	learn: 0.6265186	total: 101ms	remaining: 11.1s
9:	learn: 0.6213588	total: 105ms	remaining: 10.4s
10:	learn: 0.6148084	total: 109ms	remaining: 9.84s
11:	learn: 0.6086569	total: 113ms	remaining: 9.28s
12:	learn: 0.6032204	total: 117ms	remaining: 8.85s
13:	learn: 0.5978261	total: 122ms	remaining: 8.62s
14:	learn: 0.5937757	total: 126ms	remaining: 8.29s
15:	learn: 0.5885877	total: 130ms	remaining: 7.97s
16:	learn: 0.5846243	total: 135ms	remaining: 7.81s
17:	learn: 0.5807518	total: 139ms	remaining: 7.57s
18:	learn: 0.5768481	total: 143ms	remaining: 7.37s
19:	l

In [23]:
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import make_scorer, f1_score

cv_strategy = StratifiedKFold(n_splits=5)

scoring = ['f1', 'roc_auc']

cv_res = cross_validate(
    pipeline,
    data,
    data['target'],
    cv=cv_strategy,
    scoring=scoring
)
for key, value in cv_res.items():
    print(f'avg_{key}: {value.mean().round(2)}')

Learning rate set to 0.021508
0:	learn: 0.6843571	total: 8.55ms	remaining: 8.54s
1:	learn: 0.6764862	total: 15.3ms	remaining: 7.66s
2:	learn: 0.6677716	total: 19.9ms	remaining: 6.62s
3:	learn: 0.6592120	total: 24.5ms	remaining: 6.11s
4:	learn: 0.6523233	total: 28.4ms	remaining: 5.65s
5:	learn: 0.6456917	total: 34.6ms	remaining: 5.74s
6:	learn: 0.6392862	total: 39.7ms	remaining: 5.63s
7:	learn: 0.6332613	total: 43.4ms	remaining: 5.38s
8:	learn: 0.6267390	total: 47.5ms	remaining: 5.23s
9:	learn: 0.6215676	total: 50.7ms	remaining: 5.02s
10:	learn: 0.6153908	total: 55ms	remaining: 4.95s
11:	learn: 0.6094370	total: 58.8ms	remaining: 4.84s
12:	learn: 0.6039937	total: 62.4ms	remaining: 4.74s
13:	learn: 0.5997521	total: 66.6ms	remaining: 4.69s
14:	learn: 0.5957948	total: 71.3ms	remaining: 4.68s
15:	learn: 0.5910018	total: 74.7ms	remaining: 4.6s
16:	learn: 0.5870995	total: 78ms	remaining: 4.51s
17:	learn: 0.5832283	total: 81.4ms	remaining: 4.44s
18:	learn: 0.5802394	total: 84.9ms	remaining: 4.3